In [7]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%matplotlib inline
%pip install nltk

You should consider upgrading via the '/home/justyn/datamining/dm-finalproj/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/justyn/datamining/dm-finalproj/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/justyn/datamining/dm-finalproj/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/justyn/datamining/dm-finalproj/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer

"""
If you use the VADER sentiment analysis tools, please cite:

Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for
Sentiment Analysis of Social Media Text. Eighth International Conference on
Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.
"""
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/justyn/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [38]:
df = pd.read_csv("../resources/raw-dataset.csv")
df['row_id'] = df.index + 1
df.head()

,category,rating,label,text,row_id
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor...",1
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I...",2
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...,3
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i...",4
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...,5


In [11]:
# Make a copy of the df, incase we need to go back and access anything later
sub_df = df[['row_id', 'text']]
sub_df.head()

,row_id,text
0,1,"Love this! Well made, sturdy, and very comfor..."
1,2,"love it, a great upgrade from the original. I..."
2,3,This pillow saved my back. I love the look and...
3,4,"Missing information on how to use it, but it i..."
4,5,Very nice set. Good quality. We have had the s...


In [12]:
# Cleanup of the data
sub_df['text'] = sub_df['text'].str.replace("[^a-zA-Z#]", " ")
sub_df['text'] = sub_df['text'].str.casefold()
sub_df.head()

/tmp/ipykernel_350/2430751107.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  sub_df['text'] = sub_df['text'].str.replace("[^a-zA-Z#]", " ")
/tmp/ipykernel_350/2430751107.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['text'] = sub_df['text'].str.replace("[^a-zA-Z#]", " ")
/tmp/ipykernel_350/2430751107.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['text'] = sub_df['text'].str.casefold()


,row_id,text
0,1,love this well made sturdy and very comfor...
1,2,love it a great upgrade from the original i...
2,3,this pillow saved my back i love the look and...
3,4,missing information on how to use it but it i...
4,5,very nice set good quality we have had the s...


In [18]:
sid = SentimentIntensityAnalyzer()
words = "I like to eat candy"
scores = sid.polarity_scores(words)
print(scores)

{'neg': 0.0, 'neu': 0.545, 'pos': 0.455, 'compound': 0.3612}


In [19]:
sid = SentimentIntensityAnalyzer()
sub_df['scores'] = sub_df['text'].apply(lambda x: sid.polarity_scores(x))

/tmp/ipykernel_350/2426375197.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['scores'] = sub_df['text'].apply(lambda x: sid.polarity_scores(x))


In [ ]:
sub_df['neg'] = sub_df['scores'].apply(lambda x: x['neg'])
sub_df['neu'] = sub_df['scores'].apply(lambda x: x['neu'])
sub_df['pos'] = sub_df['scores'].apply(lambda x: x['pos'])
sub_df['compound'] = sub_df['scores'].apply(lambda x: x['compound'])
sub_df = sub_df.drop(columns=['scores'])

In [34]:
sub_df.to_csv('../resources/sentiment_scores.csv')